In [1]:
import pandas as pd
from sqlalchemy import create_engine
import json
from pandas.io.json import json_normalize
import pprint
import numpy as np
from webdriver_manager.chrome import ChromeDriverManager 

## Extract Youtube Data

In [2]:
csv_file = "./Resources/USvideos.csv"
youtube_data_df = pd.read_csv(csv_file, usecols=['video_id', 'title', 'views', 'tags'])


video_preurl = "https://www.youtube.com/watch?v="
youtube_data_df = youtube_data_df.drop_duplicates(subset="video_id", keep="last")

youtube_data_df['video_url'] = video_preurl + youtube_data_df['video_id']

youtube_data_df.head()

,video_id,title,tags,views,video_url
10,9wRQljFNDW8,Dion Lewis' 103-Yd Kick Return TD vs. Denver! ...,"NFL|""Football""|""offense""|""defense""|""afc""|""nfc""...",81377,https://www.youtube.com/watch?v=9wRQljFNDW8
36,Om_zGhJLZ5U,TL;DW - Every DCEU Movie Before Justice League,"screenjunkies|""screen junkies""|""sj news""|""hone...",288922,https://www.youtube.com/watch?v=Om_zGhJLZ5U
41,goP4Z5wyOlM,Iraq-Iran earthquake: Deadly tremor hits borde...,"bbc|""bbc news""|""news""|""iran""|""iran news""|""iraq...",34785,https://www.youtube.com/watch?v=goP4Z5wyOlM
55,8NHA23f7LvU,Jason Momoa Wows Hugh Grant With Some Dothraki...,"Graham Norton|""Graham Norton Show Official""|""E...",1496225,https://www.youtube.com/watch?v=8NHA23f7LvU
76,IE-xepGLVt8,Mayo Clinic's first face transplant patient me...,"Mayo Clinic|""Health Care (Issue)""|""Healthcare ...",237307,https://www.youtube.com/watch?v=IE-xepGLVt8


In [3]:
youtube_data_df[youtube_data_df['video_id']=='JcC5VGOx8I8']

,video_id,title,tags,views,video_url
3597,JcC5VGOx8I8,Calum Scott - You Are The Reason (Lyric Video),"Pop|""calum scott""|""you are the reason""|""you ar...",1728219,https://www.youtube.com/watch?v=JcC5VGOx8I8


## Extract TikTot Data

In [4]:
json_file = './Resources/trending.json'
with open(json_file, 'r', encoding='utf-8') as j:
     data = json.loads(j.read())

tiktok_data_df = pd.DataFrame(data['collector'], columns=['id', 'text', 'webVideoUrl', 'playCount', 'hashtags'])
tiktok_data_df.head()

,id,text,webVideoUrl,playCount,hashtags
0,6907228749016714497,Confidence went 📈,https://www.tiktok.com/@ninakleij/video/690722...,44800,[]
1,6875468410612993286,Quiet Zone... follow me on insta: joeysofo. Co...,https://www.tiktok.com/@joeysofo/video/6875468...,838100,[]
2,6898699405898059010,Iphone bend test🤗 #tiktok #viral #fyp #iphone ...,https://www.tiktok.com/@jackeyephone/video/689...,15300000,"[{'id': '23428', 'name': 'tiktok', 'title': ''..."
3,6902819837345533186,,https://www.tiktok.com/@naomivaneeren/video/69...,94900,[]
4,6905635666588192002,小技です👟✨#tiktok教室#tutorial,https://www.tiktok.com/@io.dreamer_mk/video/69...,115300,"[{'id': '1627704798586882', 'name': 'tiktok教室'..."


In [5]:
# Connect to Database 
rds_connection_string = "trends_project:123456@localhost:5432/trending_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

ModuleNotFoundError: No module named 'psycopg2'

In [ ]:
engine.table_names() 

## Transform Youtube Data 

In [ ]:
new_youtube_df = youtube_data_df.rename(columns={"title": "video_title", "views": "video_view_count"})

new_youtube_df.head()

In [ ]:
lst_col = 'name'
new_youtube_df['name'] = new_youtube_df['tags'].str.split('|')

new_youtube_df2 = pd.DataFrame({col:np.repeat(new_youtube_df[col].values, new_youtube_df[lst_col].str.len())
              for col in new_youtube_df.columns.difference([lst_col])
             }).assign(**{lst_col:np.concatenate(new_youtube_df[lst_col].values)})[new_youtube_df.columns.tolist()]

#new_youtube_df2 = youtube_data_df.rename(columns={"video_id": "platform_id", "views": "view_count", "split_tags":"name"})

new_youtube_df2 = new_youtube_df2.drop(columns=['tags'])
new_youtube_df2.head()

In [ ]:
test_tiktok_data_df = pd.json_normalize(data['collector'], record_path="hashtags", meta=['id', 'text', 'webVideoUrl', 'playCount'], meta_prefix='video_')
test_tiktok_data_df = test_tiktok_data_df.drop(columns=['id','title','cover'])
test_tiktok_data_df

## Transform Tiktok Data

In [ ]:
test_tiktok_data_df = test_tiktok_data_df.rename(columns={"video_text": "video_title", "video_webVideoUrl": "video_url", "video_playCount": "video_view_count"})
test_tiktok_data_df.head()

### Merge Data

In [ ]:
combined_tags_df = test_tiktok_data_df.append(new_youtube_df2)
combined_tags_df.head()

## Load Data 

In [ ]:
combined_tags_df.to_sql('tags', if_exists='replace', con=engine, index=False) 
#engine.execute('select * from tags').fetchall()

In [ ]:
tagcount = combined_tags_df['name'].nunique()
print(f'Total unique tags: {tagcount:,d}')

In [ ]:
videocount = combined_tags_df['video_id'].nunique()
print(f'Total unique videos: {videocount:,d}')

# Analyze Data !

In [ ]:
summarydf = combined_tags_df.groupby(['name'])['video_view_count'].sum().reset_index()
summarydf.sort_values('video_view_count', ascending=False)

## Top 10 Tags

In [ ]:
summarydf.sort_values('video_view_count', ascending=False).head(10)

## Bottom 10 Tags

In [ ]:
summarydf.sort_values('video_view_count', ascending=False).tail(10)

## Top 10 Videos by "Pop"

In [ ]:
def make_clickable(val):
    return '<a href="{}">{}</a>'.format(val,val)

In [ ]:
popdf = combined_tags_df[combined_tags_df['name'] == "\"Pop\""]
popdf.head(10).style.format({'video_url': make_clickable})

## Top 10 Videos by "funny"

In [ ]:
funnydf = combined_tags_df[combined_tags_df['name'] == "\"funny\""]
funnydf.head(10).style.format({'video_url': make_clickable})